<center>

## MAGICIAN: Mining and gAmblinG servIces sCrapIng and ANalysis

Matteo Martini 636694

In [1]:
mining_pool_url = "https://en.bitcoin.it/wiki/DeepBit9"


#Individuare nel DataSet tutte le transazioni relative a Deepbit.net,ovvero le transazioni che presentano almeno un indirizzo di input o di output appartenente a Deepbit_addrs.
#servizio scommesse = DiceOnCrack

## Setup Ambiente

In [75]:
# Librerie concesse
import pandas as pd

# Per scraping (scegline uno o entrambi, in base alle esigenze delle pagine)
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Per formati efficienti (PyArrow)
import pyarrow as pa
import pyarrow.parquet as pq

# Utility
from pathlib import Path
from datetime import datetime

import numpy as np


from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
import time
import requests
import re
from bs4 import BeautifulSoup

# Percorsi (aggiorna con il tuo percorso locale)
DATA_DIR = Path('./Datasets')   # metti qui i CSV scaricati

PARQUET_DIR = Path('./data_parquet')
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
print('Environment ready.')

Environment ready.


## 1) Costruzione dei Dataframe dal Dataset

## Directory Datasets


In [76]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

## Definizone Data Type
Sapendo cosa contengono i file csv, possiamo forzare la conversione in determiinati tipi per ridurre l'impatto in memoria dei dataset

In [83]:
dtype_transactions = {
    "timestamp": np.int32,          # prima colonna: secondi (1231469xxx) rientra in int32
    "tx_id": np.int32,              # seconda colonna: contatore 1…10 572 827
    "block_height": np.int32,       # terza colonna: 1…214 562
    "prev_tx_id": np.int32,         # quarta colonna: seguono il tx_id precedente
    "has_witness": np.uint8,        # quinta colonna 0/1
    "fee": np.int32,                # sesta colonna: 0…100 000 (int32 più che sufficiente)
}


dtype_inputs = {
    "input_id": np.int32,
    "tx_id": np.int32,
    "prev_output_id": np.int32,
    "input_index": np.uint8,
}


dtype_outputs = {
    "col0":np.int32,
    "col1":np.int32,
    "col2":np.int8,
    "col3":np.int32,
    "col4":np.int64,
    "col5":"uint8"
}

dtype_mapping = {
    "address": "category",   # stringhe ripetute → category riduce drasticamente
    "address_id": "Int32",   # valori 0–8 708 820 entrano in 32 bit
}


columns_inputs = [
    "tx_id",
    "prev_tx_id",
    "prev_tx_pos"
]
columns_transactions = [
    "timestamp", "tx_id", "block_height",
     "has_witness", "fee"
]
columns_outputs = [
    "tx_id","position",
    "addressId","amount","script_type"
]
columns_mapping = [
    "hash", "addressId"
]

## Caricamento dei Dataset

In [84]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

transactions = pd.read_csv(transactions_csv,header=None,names=columns_transactions,dtype=dtype_transactions)
inputs_df    = pd.read_csv(inputs_csv,header=None,names=columns_inputs,dtype=dtype_inputs)
outputs_df   = pd.read_csv(outputs_csv,header=None,names=columns_outputs,dtype=dtype_outputs)
mapping_df   = pd.read_csv(mapping_csv,header=None,names=columns_mapping,dtype=dtype_mapping)

## Operazioni su Dataset

In [ ]:
display(inputs_df)
display(outputs_df)
display(mapping_df)


display(transactions)
merged_outputs = outputs_df.merge(mapping_df,on="addressId",how="left")

,tx_id,prev_tx_id,prev_tx_pos
0,171,9,0
1,183,171,1
2,185,183,1
3,187,185,1
4,192,187,0
...,...,...,...
21378765,10572823,10572820,7
21378766,10572824,10572820,5
21378767,10572825,10572820,6
21378768,10572826,10572820,2


,tx_id,position,addressId,amount,script_type
0,0,0,0,5000000000,1
1,1,0,1,5000000000,1
2,2,0,2,5000000000,1
3,3,0,3,5000000000,1
4,4,0,4,5000000000,1
...,...,...,...,...,...
24613794,10572825,0,8707915,1,2
24613795,10572825,1,6137715,9899999,2
24613796,10572826,0,8707915,26651306,2
24613797,10572826,1,6137711,7991620447,2


,hash,addressId
0,1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa,0
1,12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX,1
2,1HLoD9E4SDFFPDiYfNYnkBLQ85Y51J3Zb1,2
3,1FvzCLoTPGANNjWoUo6jUGuAG3wg1w4YjR,3
4,15ubicBBWFnvoZLT7GiU2qxjRaKJPdkDMG,4
...,...,...
8708816,13dJEWNK55L5Zbh8ZVqpvmpritkEYHD1nE,8708816
8708817,1GA1vXcj4UMwcgr7nBftkpxXE6RrBwwm2M,8708817
8708818,1DrJbVsNJDweTQgKAwrBNyKuvtFBS6iKNK,8708818
8708819,1Mtcwd8jsoMYdwBxcybBkFWm1NNCRMRToh,8708819


,timestamp,tx_id,block_height,has_witness,fee
0,1231006505,0,0,1,0
1,1231469665,1,1,1,0
2,1231469744,2,2,1,0
3,1231470173,3,3,1,0
4,1231470988,4,4,1,0
...,...,...,...,...,...
10572824,1356997957,214562,10572822,0,100000
10572825,1356997957,214562,10572823,0,100000
10572826,1356997957,214562,10572824,0,100000
10572827,1356997957,214562,10572825,0,100000


In [87]:
display(merged_outputs)

,tx_id,position,addressId,amount,script_type,hash
0,0,0,0,5000000000,1,1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa
1,1,0,1,5000000000,1,12c6DSiU4Rq3P4ZxziKxzrL5LmMBrzjrJX
2,2,0,2,5000000000,1,1HLoD9E4SDFFPDiYfNYnkBLQ85Y51J3Zb1
3,3,0,3,5000000000,1,1FvzCLoTPGANNjWoUo6jUGuAG3wg1w4YjR
4,4,0,4,5000000000,1,15ubicBBWFnvoZLT7GiU2qxjRaKJPdkDMG
...,...,...,...,...,...,...
24613794,10572825,0,8707915,1,2,1DYve98WqJQmx6cDdW2EQFpnqfwptjV4e9
24613795,10572825,1,6137715,9899999,2,1AdN2my8NxvGcisPGYeQTAKdWJuUzNkQxG
24613796,10572826,0,8707915,26651306,2,1DYve98WqJQmx6cDdW2EQFpnqfwptjV4e9
24613797,10572826,1,6137711,7991620447,2,1L4EThM6x3Rd2PjNbs1U136FpMq4Gmo3fJ


## 2) Recupero informazioni da WalletExplorer


### Definizione Funzioni Helper per Scraping

In [9]:
wallet_explorer_url = "https://www.walletexplorer.com/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/127.0.0.1 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.walletexplorer.com/",
}

In [ ]:
def next_page(driver):
    #cerco il link alla prossima pagina
    for _ in ("Next","Last"):
        try:
            el = driver.find_element(By.XPATH, f'//div[@class="paging"]//a[contains(., "{_}")]')
            url = el.get_attribute("href")
            el.click()
            return url
        except:
            return None
        
# def find_next_link(driver,element):
#     try:
#         el = driver.find_element(By.XPATH, f'//div[@class="paging"]//a[contains(., "{element}")]')
#         url = el.get_attribute("href")
#         el.click()
#         return url
#     except:
#         return None

# def get_transactions_link(soup,class_):
#     transactions = soup.find_all('tr',class_=class_)
#     page_links = []
#     for trans in transactions:
#         txid = trans.find('td',class_="txid").find('a')['href']
#         page_links.append(txid)
#     return page_links

# def safe_get(session, url, tries=3, delay=1):
    for attempt in range(tries):
        try:
            return session.get(url, timeout=10)
        except (requests.exceptions.ConnectionError,
                requests.exceptions.RemoteDisconnectedError):
            if attempt == tries - 1:
                raise
            time.sleep(delay * (attempt + 1))  # 1s, poi 2s, poi 3s…

def scrape_step(driver):
    #recupero le entries
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # seleziona tutti gli elementi che hanno l’attributo data-wallet-id
    wallet_elements = soup.select('[data-wallet-id]')
    # estrai il valore dell’attributo
    wallet_ids = [el['data-wallet-id'] for el in wallet_elements]
    #vado alla prossima pagina
    return wallet_ids




### Definizione Funzione per Scraping

In [ ]:
def scrape_wallet_explorer(wallet_name):
    driver = webdriver.Chrome()
    driver.get(wallet_explorer_url)
    
    #ricerca del wallet tramite form
    driver.find_element(By.NAME,'q').send_keys(wallet_name)
    driver.find_element(By.TAG_NAME,'form').submit()
    
    #mettere funzione di selenium
    time.sleep(1)

    try:
        while True:
            #recupero di tutti gli input/output della pagina
            yield from scrape_step(driver)

            if next_page(driver) == None:break;
    except:
            print('Something went wrong')
    finally:
        driver.close()    
   


In [109]:
bw = list(scrape_wallet_explorer("BW"))
print(bw)

[<a href="/wallet/00691e47cf198f0d"><span class="wallet_color" style="background-color: #198f0d"></span><span class="wallet_name">[00691e47cf]</span></a>, <a href="/wallet/00691e47cf198f0d"><span class="wallet_color" style="background-color: #198f0d"></span><span class="wallet_name">[00691e47cf]</span></a>, <a href="/wallet/4e1fd37c1dbd4eb6"><span class="wallet_color" style="background-color: #bd4eb6"></span><span class="wallet_name">[4e1fd37c1d]</span></a>, <a href="/wallet/4367c2591d88b609"><span class="wallet_color" style="background-color: #88b609"></span><span class="wallet_name">[4367c2591d]</span></a>, <a href="/wallet/ChBtc.com"><span class="wallet_color" style="background-color: #63d83e"></span><span class="wallet_name">ChBtc.com</span></a>, <a href="/wallet/00abb03a72d2cf3d"><span class="wallet_color" style="background-color: #d2cf3d"></span><span class="wallet_name">[00abb03a72]</span></a>, <a href="/wallet/ChBtc.com"><span class="wallet_color" style="background-color: #63d8

In [110]:
print(bw)
len(bw)

['/wallet/00691e47cf198f0d', '/wallet/00691e47cf198f0d', '/wallet/4e1fd37c1dbd4eb6', '/wallet/4367c2591d88b609', '/wallet/ChBtc.com', '/wallet/00abb03a72d2cf3d', '/wallet/ChBtc.com', '/wallet/00abb03a72d2cf3d', '/wallet/ChBtc.com', '/wallet/00abb03a72d2cf3d', '/wallet/00abb03a72d2cf3d', '/wallet/ChBtc.com', '/wallet/ChBtc.com', '/wallet/00abb03a72d2cf3d', '/wallet/ChBtc.com', '/wallet/ChBtc.com', '/wallet/00abb03a72d2cf3d', '/wallet/00abb03a72d2cf3d', '/wallet/00abb03a72d2cf3d', '/wallet/0095382c3ef6cc3b', '/wallet/00abb03a72d2cf3d', '/wallet/00abb03a72d2cf3d', '/wallet/00abb03a72d2cf3d', '/wallet/00abb03a72d2cf3d', '/wallet/0a12d4d5b7b9cf33', '/wallet/00abb03a72d2cf3d', '/wallet/0a12d4d5b7b9cf33', '/wallet/00abb03a72d2cf3d', '/wallet/00abb03a72d2cf3d', '/wallet/00abb03a72d2cf3d', '/wallet/00abb03a72d2cf3d', '/wallet/0a12d4d5b7b9cf33', '/wallet/0a12d4d5b7b9cf33', '/wallet/0a12d4d5b7b9cf33', '/wallet/0a12d4d5b7b9cf33', '/wallet/0a12d4d5b7b9cf33', '/wallet/0a12d4d5b7b9cf33', '/wallet/0a1

470

### Probabilmente non serve ma si cancella solo a progetto finito

In [ ]:
# BLOCK_RE = re.compile(r"\d+")
# FEE_RE = re.compile(r"\d+(?:\.\d+)?")

# def get_info_from_transaction_table(trans_table_link,session):
#     # res = session.get(f'{"https://www.walletexplorer.com"}{trans_table_link}',timeout=10)
#     res = safe_get(session,f'{"https://www.walletexplorer.com"}{trans_table_link}')
#     soup = BeautifulSoup(res.text,"html.parser")
#     cells = soup.select('table.info td')
#     all_txt = [td.get_text() for td in cells]
#     match  = BLOCK_RE.search(all_txt[1])
#     if match:blockId = int(match.group())
#     else:   blockId =  None
#     match_2 = FEE_RE.search(all_txt[4]) 
#     if match_2: fee = float(match_2.group())  
#     else: fee= None
#     return {
#         "timestamp":all_txt[2],
#         "blockId":blockId,
#         "txId":all_txt[0],
#         "isCoinbase":0,
#         "fee":fee
#     }


# def get_wallet_adress(trans_table_link,session):
#     res = safe_get(session,f'{"https://www.walletexplorer.com"}{trans_table_link}')
#     soup = BeautifulSoup(res.text,"html.parser")
#     cells = soup.select('table.info td')
#     all_txt = [td.get_text() for td in cells]
#     return all_txt[3]

# def get_all_trans(transaction_list):
#     infos = []
#     session = requests.Session()
#     session.headers.update(headers)
#     for transaction_link in transaction_list:
#         # info = get_info_from_transaction_table(transaction_link,session)
#         info = get_wallet_adress(transaction_link,session)
#         infos.append(info)
#     return infos

<center>

# Analisi Transazioni DiceOnCrack


In [ ]:
diceoncrack_transaction = list(scrape_wallet_explorer('DiceonCrack'))
print(len(diceoncrack_transaction))

91321


In [48]:
display(diceoncrack_transaction)

['00cf1613d74c513e',
 '99df98abccf37c07',
 'c2007ccf096138f9',
 '77bebf6682d16a66',
 '3383c3c7d2d376c4',
 '8aa9b2e64b8d83dd',
 '7d90100a1ac56ced',
 'ec9ac79b3a4d00fe',
 '11fe3742882d50cc',
 '5c814aaefb1bcbe4',
 'f24f62123c567576',
 '166ee82c89ad40c4',
 '166ee82c89ad40c4',
 'c9dffbdf043464c7',
 '0008a70ca8bbe3c5',
 '0008a70ca8bbe3c5',
 '0008a70ca8bbe3c5',
 '3b7aff74efe08f94',
 'add44697f4dbbde7',
 '8ff0a5158cb525b3',
 '013b2a52a8979274',
 '41aa1cd065d5fe37',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '5f2b26ed990121c2',
 '5f2b26ed990121c2',
 '00a376a54df460fe',
 'c473658a837249a8',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '022d9ac4502b9ccb',
 '022d9ac4502b9ccb',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '1172596d83a16e70',
 '002c5c9ea45763f9',
 '002c5c9ea45763f9',
 '002c5c9ea45763f9',
 '0c4f1c38b96d10ac',
 '002c5c9ea45763f9',
 '0c4f1c38b96d10ac',
 'd5543daeccdca46e',
 '002c5c9ea45763f9',
 '002c5c9ea45

In [69]:
bw = list(scrape_wallet_explorer('BW'))

### Salvataggio delle transazioni in Csv per velocizzare il testing

In [88]:
# salva il DataFrame in un CSV nella cartella corrente
doc_df = pd.DataFrame(diceoncrack_transaction,columns=["hash"]).set_index("hash")
doc_df.to_csv("diceoncrack.csv", index=True, encoding="utf-8")

### Recupero dei dati dal csv

In [100]:
dice_df = pd.read_csv('diceoncrack.csv',index_col="hash")
dice_set = set(dice_df.index.to_list())
print(dice_set)

# display(merged_outputs)

dice_outputs = merged_outputs[merged_outputs["hash"] == "148hw7FzSVABsvdsfZFER2xhgAUVdtri7s"]
display(dice_outputs)

{'377278e1a3767acc', '3ac9b4ea05690432', '7d9cec15916d54d3', 'ce8fcd2814d394b2', 'fd375b5adb40dd96', '66aed70aef9c4509', '924765c4e1f582a9', 'b27ea57ba66b7e75', '9505d73a5fa7c034', '000000030ae8727e', 'f2079aebd9c2ad31', 'f181d4002ceb0bea', '9c0320b28c54acc6', '4a2a5112b07c9702', 'e8e45dd7d50d16ea', '5f01841f8138bf60', '0d0b341d4eb1d40c', '50499de224ba6eef', '957aacf6a7430de4', '371111b25b65ef5f', '42a3e3261a59ed61', '6c871b1abcfc972a', 'e8adc6dbd7ae0229', 'c2ad66f6c260e581', '7166876b00e08a0b', 'c2370a9dd76ae765', '07425348e6c92d16', '67757831a65d5094', 'e590f17ab87f4374', '150fe422181e65d0', '3bd23f9506746b58', '8b4162fc199f1928', '979d38fb0acece47', '7c3b0143cfa1d031', 'b5badb56d3400e8e', '188195f2db91e1cd', '67b0c41a02035203', '2d8b7cf11635a3f0', '52d7eb725a47603b', '23aec21e69ef9c78', '9bb3d260c2db29a4', '459885d4c0c053b1', '210cc68a75b80675', '1addbbbe45fd3ebf', 'fe525f4084a2c7d5', 'ae3e8719e50b8abc', '3a1d225a5c158a1b', '89d2a05337ba3c75', '3d127212e132c671', '02954ddbc6411361',

,tx_id,position,addressId,amount,script_type,hash


In [ ]:
dice_addesses_df = pd.merge(dice_df,merged_outputs,on="hash",how="inner")
#00cf1613d74c513e
#1A1zP1eP5QGefi2DMPTfTL5SLmv7DivfNa
display(dice_addesses_df)

,hash,tx_id,position,addressId,amount,script_type


<center>

# Analisi Transazioni di DeepBit.Net

In [72]:
deepbit_transactions = list(scrape_wallet_explorer('DeepBit.net'))
print(len(deepbit_transactions))

794767


In [73]:
print(len(deepbit_transactions))
display(deepbit_transactions)

794767


['bec2e851437364e6',
 '7ac06a3f716a3a6d',
 'c3e8a5a6854cee45',
 '7abd23ecf9177763',
 '234dd1317c3c2981',
 '492c37c7da55714a',
 '909c1f1a4b3e1f2f',
 '2e4c6685b8650662',
 '2e4c6685b8650662',
 '03b368c35dc78812',
 '43969dbba2f05c5d',
 'de751d7d4452c067',
 'e5ddf53645ef16c2',
 'fa78ebe5bcf0b5c2',
 'ad1352521dd0e9c8',
 '880878ab85cdbf22',
 '010fec42785f07d9',
 '31fc9650564bf9e9',
 'cbb3ac40291b5ebf',
 'af35bf3bdedb5ce6',
 '3250b4851e3c9dd4',
 '8d7cfbcc2c8a8085',
 'b94b0e9be6baf74f',
 '6b87b502641304d6',
 'e4a9aba67d1a3323',
 '8a38b3ab3d65dd57',
 '179c34245206f652',
 '39ef8420d1a1df3b',
 '2b74cd7f65f036d8',
 '7dd10109a1fe444b',
 '3553747318154de5',
 '0008a70ca8bbe3c5',
 '0008a70ca8bbe3c5',
 '0008a70ca8bbe3c5',
 '02cea279375aa532',
 'bfc7477dc42a4b40',
 '02cea279375aa532',
 '02cea279375aa532',
 '04392b677775252f',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938b24b9',
 '004d9e07938

In [74]:
# salva il DataFrame in un CSV nella cartella corrente
doc_df = pd.DataFrame(deepbit_transactions,columns=["inputs/outputs"]).set_index("inputs/outputs")
doc_df.to_csv("deepbit.csv", index=True, encoding="utf-8")

In [ ]:
deepbit_df = pd.read_csv('deepbit.csv',index_col="inputs/outputs")